In [1]:
pip install -U ipython

In [2]:
!pip install autokeras
!pip install tensorboardX
!pip install mosaicml
import tensorflow as tf
import autokeras as ak
from keras.datasets import cifar10

from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
import composer.functional as cf

Using TensorFlow backend


In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# Take only 5000 samples from the training set
num_samples = 5000
train_images = train_images[:num_samples]
train_labels = train_labels[:num_samples]

# Take only 5000 samples from the test set
test_images = test_images[:num_samples]
test_labels = test_labels[:num_samples]

# The 'train_images', 'train_labels', 'test_images', and 'test_labels' now contain only 5000 samples each
# Normalize the pixel values to the range [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0


In [4]:
clf = ak.ImageClassifier(
    max_trials=10,  # Number of different models to try
    objective='val_accuracy',  # Metric to optimize
    seed=42,  # Seed for reproducibility
)


Reloading Tuner from ./image_classifier/tuner0.json


In [5]:
clf.fit(train_images, train_labels, epochs=2, validation_split=0.2, batch_size= 16)


Epoch 1/2
313/313 [==============================] - 419s 829ms/step - loss: 1.6624 - accuracy: 0.5120
Epoch 2/2
313/313 [==============================] - 262s 838ms/step - loss: 0.6821 - accuracy: 0.8024


In [8]:
tuner = clf.tuner
trial_ids = tuner.oracle.trials
trials_hparams=[]
score=[]
for trial_id in trial_ids:
    trial = clf.tuner.oracle.get_trial(trial_id)
    trial.best_step
    trial.metrics
    hparams = trial.hyperparameters.values
    score.append(trial.score)
  # Append to list
    trials_hparams.append(hparams)


import pandas as pd
df_bayesian = pd.DataFrame(trials_hparams)
df_bayesian['val_Loss']=score
df_bayesian

,image_block_1/normalize,image_block_1/augment,image_block_1/block_type,classification_head_1/spatial_reduction_1/reduction_type,classification_head_1/dropout,optimizer,learning_rate,image_block_1/image_augmentation_1/translation_factor,image_block_1/image_augmentation_1/horizontal_flip,image_block_1/image_augmentation_1/vertical_flip,...,image_block_1/conv_block_1/max_pooling,image_block_1/conv_block_1/separable,image_block_1/conv_block_1/dropout,image_block_1/conv_block_1/filters_0_0,image_block_1/conv_block_1/filters_0_1,image_block_1/conv_block_1/filters_1_0,image_block_1/conv_block_1/filters_1_1,image_block_1/conv_block_1/filters_2_0,image_block_1/conv_block_1/filters_2_1,val_Loss
0,True,True,efficient,global_max,0.0,adam,0.00002,0.1,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.489837
1,True,True,resnet,global_avg,0.0,adam,0.00002,0.1,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.105691
2,True,False,vanilla,flatten,0.5,adam,0.00100,NaN,NaN,NaN,...,True,False,0.25,32.0,64.0,NaN,NaN,NaN,NaN,0.466942
3,True,True,efficient,global_avg,0.0,adam,0.00002,0.1,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.778926
4,True,True,resnet,global_avg,0.0,adam,0.00100,0.1,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.220041
5,False,True,efficient,global_avg,0.0,adam,0.00002,0.1,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.448171
6,True,True,vanilla,global_avg,0.0,adam,0.00002,0.1,True,False,...,False,False,0.50,512.0,16.0,32.0,32.0,32.0,32.0,0.168699
7,True,True,efficient,global_avg,0.0,adam,0.00002,0.1,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.163618
8,True,True,efficient,global_avg,0.0,adam,0.00002,0.1,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.765496
9,True,True,efficient,global_avg,0.0,adam,0.10000,0.1,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.102273


In [10]:
df_bayesian.to_csv('architechtures.csv')